In [1]:
import pandas as pd
train=pd.read_csv('IndoreHP_Train.csv')
test=pd.read_csv('IndoreHP_Test.csv')

X=train.drop(columns=['MEDV'])
y=train['MEDV']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=42)


from sklearn.preprocessing import MinMaxScaler
# Perform Min-Max scaling on the features
scaler = MinMaxScaler(feature_range=(0,1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
test=scaler.transform(test)

In [4]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer

train_data=lgb.Dataset(X_train,label=y_train)
lgb_estimator=lgb.LGBMRegressor()

param_grid={
    'objective':['quantile'],
    'alpha':[0.3,0.5,0.7],
    'learning_rate':(0.05,1.5),
    'num_leaves':(3,150),
    'min_child_samples':(3,50),
    'verbosity':[-1],
}

search=GridSearchCV(
    lgb_estimator,
    param_grid,
    scoring=make_scorer(mean_absolute_error,greater_is_better=False),
    cv=5,
    verbose=0
)

In [5]:
search.fit(X_train,y_train)
best_params=search.best_params_
print(best_params)

{'alpha': 0.5, 'learning_rate': 0.05, 'min_child_samples': 3, 'num_leaves': 150, 'objective': 'quantile', 'verbosity': -1}


In [6]:
lgb_model=lgb.train(best_params,train_data,num_boost_round=200)
y_pred = lgb_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Validation MAE:", mae)

Validation MAE: 2.22738798398294


In [7]:
y_pred1=lgb_model.predict(test)

submission=pd.read_csv('IndoreHP_Sample.csv')
submission['MEDV']=y_pred1
submission.to_csv('./submission2.csv', index=False)
submission.head()

,ID,MEDV
0,400,20.540293
1,401,25.140606
2,402,30.006677
3,403,21.411138
4,404,28.567649
